# 🗑️ Identify Redundant Playlists & Reorganization

Analyze your playlists to identify **redundant playlists** that can be safely deleted or merged without losing information.

**What this notebook does:**
- 🔍 Finds playlists with high track overlap (duplicates/near-duplicates)
- 📊 Identifies playlists that are subsets of other playlists
- 🎯 Suggests playlists safe to delete
- 📋 Proposes a reorganized library structure
- 💡 Recommends consolidation strategies

**Prerequisites:** Run `01_sync_data.ipynb` first to download your library.


## 1️⃣ Setup


In [ ]:
# Install dependencies
%pip install -q pandas pyarrow tqdm

# Add project to path
import sys
from pathlib import Path
PROJECT_ROOT = Path("..").resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print(f"✅ Project root: {PROJECT_ROOT}")


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from collections import defaultdict, Counter
from typing import Dict, List, Set, Tuple
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

from spotim8.analysis import LibraryAnalyzer, PlaylistSimilarityEngine

# Data directory
DATA_DIR = PROJECT_ROOT / "data"
print(f"📁 Data directory: {DATA_DIR.resolve()}")


## 2️⃣ Load Library Data


In [ ]:
# Load all data (owned playlists only for analysis)
analyzer = LibraryAnalyzer(DATA_DIR).load()

# Get owned playlists
playlists = analyzer.playlists_all[analyzer.playlists_all['is_owned'] == True].copy()
playlist_tracks = analyzer.playlist_tracks_all[
    analyzer.playlist_tracks_all['playlist_id'].isin(playlists['playlist_id'])
].copy()

print(f"✅ Loaded {len(playlists)} owned playlists")
print(f"📊 Total playlist-track links: {len(playlist_tracks):,}")

# Exclude Liked Songs from redundancy analysis (it's the master playlist)
liked_id = analyzer.liked_songs_id
if liked_id:
    playlists = playlists[playlists['playlist_id'] != liked_id].copy()
    print(f"   (Excluded Liked Songs from analysis)")

print(f"\n📋 Analyzing {len(playlists)} playlists for redundancy...")


## 3️⃣ Build Playlist Track Sets


In [ ]:
# Build track sets for each playlist
playlist_track_sets: Dict[str, Set[str]] = {}
playlist_info = {}

for pid in tqdm(playlists['playlist_id'], desc="Building track sets"):
    tracks = set(playlist_tracks[playlist_tracks['playlist_id'] == pid]['track_id'].unique())
    playlist_track_sets[pid] = tracks
    
    info = playlists[playlists['playlist_id'] == pid].iloc[0]
    playlist_info[pid] = {
        'name': info.get('name', 'Unknown'),
        'track_count': len(tracks),
        'is_liked_songs': info.get('is_liked_songs', False),
    }

print(f"✅ Built track sets for {len(playlist_track_sets)} playlists")
print(f"📊 Total unique tracks across all playlists: {len(set().union(*playlist_track_sets.values())):,}")


## 4️⃣ Find Redundant Playlists

We'll identify redundancy using multiple criteria:
1. **Exact duplicates** - Same tracks
2. **Subsets** - All tracks in one playlist are in another
3. **High overlap** - Very similar track sets (>90% overlap)
4. **Near-duplicates** - High similarity with minor differences


In [ ]:
def jaccard_similarity(set1: Set, set2: Set) -> float:
    """Calculate Jaccard similarity (intersection / union)."""
    if not set1 and not set2:
        return 1.0
    if not set1 or not set2:
        return 0.0
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union if union > 0 else 0.0

def overlap_ratio(set1: Set, set2: Set) -> Tuple[float, float]:
    """Calculate overlap ratios in both directions.
    
    Returns:
        (ratio of set1 in set2, ratio of set2 in set1)
    """
    if not set1 or not set2:
        return (0.0, 0.0)
    intersection = len(set1 & set2)
    return (intersection / len(set1), intersection / len(set2))

# Find redundant playlists
redundant_groups = []
exact_duplicates = []
subsets = []  # (subset_playlist, superset_playlist)
high_overlap = []  # (>90% similarity)
near_duplicates = []  # (80-90% similarity)

playlist_ids = list(playlist_track_sets.keys())

print("🔍 Analyzing playlist pairs...")
for i in tqdm(range(len(playlist_ids)), desc="Comparing playlists"):
    pid1 = playlist_ids[i]
    set1 = playlist_track_sets[pid1]
    
    if not set1:  # Skip empty playlists
        continue
    
    for j in range(i + 1, len(playlist_ids)):
        pid2 = playlist_ids[j]
        set2 = playlist_track_sets[pid2]
        
        if not set2:  # Skip empty playlists
            continue
        
        # Check for exact duplicates
        if set1 == set2:
            exact_duplicates.append((pid1, pid2))
            continue
        
        # Check for subsets
        if set1.issubset(set2):
            subsets.append((pid1, pid2, len(set1), len(set2)))
        elif set2.issubset(set1):
            subsets.append((pid2, pid1, len(set2), len(set1)))
        
        # Calculate similarity
        jaccard = jaccard_similarity(set1, set2)
        overlap1, overlap2 = overlap_ratio(set1, set2)
        
        if jaccard > 0.9:
            high_overlap.append((pid1, pid2, jaccard, overlap1, overlap2))
        elif jaccard > 0.8:
            near_duplicates.append((pid1, pid2, jaccard, overlap1, overlap2))

print(f"\n✅ Analysis complete!")
print(f"   Exact duplicates: {len(exact_duplicates)}")
print(f"   Subsets: {len(subsets)}")
print(f"   High overlap (>90%): {len(high_overlap)}")
print(f"   Near duplicates (80-90%): {len(near_duplicates)}")


In [ ]:
# Display exact duplicates
if exact_duplicates:
    print("=" * 80)
    print("🔄 EXACT DUPLICATES (Same tracks, can delete one)")
    print("=" * 80)
    for pid1, pid2 in exact_duplicates:
        info1 = playlist_info[pid1]
        info2 = playlist_info[pid2]
        print(f"\n📋 {info1['name']} ({info1['track_count']} tracks)")
        print(f"   ⚡ Duplicate of: {info2['name']} ({info2['track_count']} tracks)")
        print(f"   💡 Recommendation: Delete one (keep the one with better name)")
else:
    print("✅ No exact duplicates found")


In [ ]:
# Display subsets (playlists fully contained in others)
if subsets:
    print("\n" + "=" * 80)
    print("📦 SUBSETS (Fully contained in another playlist - SAFE TO DELETE)")
    print("=" * 80)
    
    subset_df = []
    for subset_pid, superset_pid, subset_size, superset_size in subsets:
        subset_info = playlist_info[subset_pid]
        superset_info = playlist_info[superset_pid]
        subset_df.append({
            'Subset Playlist': subset_info['name'],
            'Subset Tracks': subset_size,
            'Contained In': superset_info['name'],
            'Superset Tracks': superset_size,
            'Coverage': f"{subset_size}/{superset_size} ({100*subset_size/superset_size:.1f}%)"
        })
    
    df = pd.DataFrame(subset_df)
    df = df.sort_values('Subset Tracks', ascending=False)
    print(f"\n📊 Found {len(df)} playlists that are subsets of others:\n")
    print(df.to_string(index=False))
    
    # Identify unique subset playlists (safe to delete)
    safe_to_delete = set([subset_pid for subset_pid, _, _, _ in subsets])
    print(f"\n✅ {len(safe_to_delete)} playlists can be safely deleted (all tracks are in other playlists)")
else:
    print("\n✅ No subset playlists found")


In [ ]:
# Display high overlap playlists
if high_overlap:
    print("\n" + "=" * 80)
    print("🎯 HIGH OVERLAP (>90% similarity - Likely redundant)")
    print("=" * 80)
    
    overlap_df = []
    for pid1, pid2, jaccard, overlap1, overlap2 in high_overlap:
        info1 = playlist_info[pid1]
        info2 = playlist_info[pid2]
        overlap_df.append({
            'Playlist 1': info1['name'],
            'Tracks 1': info1['track_count'],
            'Playlist 2': info2['name'],
            'Tracks 2': info2['track_count'],
            'Similarity': f"{jaccard*100:.1f}%",
            'P1 in P2': f"{overlap1*100:.1f}%",
            'P2 in P1': f"{overlap2*100:.1f}%",
        })
    
    df = pd.DataFrame(overlap_df)
    df = df.sort_values('Similarity', ascending=False)
    print(f"\n📊 Found {len(df)} playlist pairs with >90% similarity:\n")
    print(df.to_string(index=False))
    
    print("\n💡 Recommendations:")
    print("   - If one playlist is much smaller, consider merging into the larger one")
    print("   - If playlists serve different purposes, keep both but remove duplicate tracks")
else:
    print("\n✅ No high-overlap playlists found")


In [ ]:
# Display near-duplicates
if near_duplicates:
    print("\n" + "=" * 80)
    print("🔗 NEAR-DUPLICATES (80-90% similarity - Review for consolidation)")
    print("=" * 80)
    
    near_df = []
    for pid1, pid2, jaccard, overlap1, overlap2 in near_duplicates[:20]:  # Show top 20
        info1 = playlist_info[pid1]
        info2 = playlist_info[pid2]
        near_df.append({
            'Playlist 1': info1['name'],
            'Tracks 1': info1['track_count'],
            'Playlist 2': info2['name'],
            'Tracks 2': info2['track_count'],
            'Similarity': f"{jaccard*100:.1f}%",
            'P1 in P2': f"{overlap1*100:.1f}%",
            'P2 in P1': f"{overlap2*100:.1f}%",
        })
    
    df = pd.DataFrame(near_df)
    df = df.sort_values('Similarity', ascending=False)
    print(f"\n📊 Top {len(df)} near-duplicate pairs (showing first 20):\n")
    print(df.to_string(index=False))
    
    if len(near_duplicates) > 20:
        print(f"\n   ... and {len(near_duplicates) - 20} more pairs")
    
    print("\n💡 Recommendations:")
    print("   - Review these pairs manually")
    print("   - Consider merging if they serve the same purpose")
    print("   - Keep separate if they have distinct purposes")
else:
    print("\n✅ No near-duplicate playlists found")


## 6️⃣ Comprehensive Redundancy Analysis

Now let's identify ALL playlists that can be safely deleted or consolidated.


In [ ]:
# Build comprehensive list of playlists safe to delete
safe_to_delete = set()
consolidation_suggestions = []

# 1. Exact duplicates - keep the first one, delete others
for pid1, pid2 in exact_duplicates:
    # Keep the one with more descriptive name or more tracks
    info1 = playlist_info[pid1]
    info2 = playlist_info[pid2]
    if len(info1['name']) >= len(info2['name']):
        safe_to_delete.add(pid2)
        consolidation_suggestions.append({
            'action': 'delete',
            'playlist_id': pid2,
            'playlist_name': info2['name'],
            'reason': f'Exact duplicate of "{info1["name"]}"',
            'tracks_lost': 0,
            'alternative': info1['name']
        })
    else:
        safe_to_delete.add(pid1)
        consolidation_suggestions.append({
            'action': 'delete',
            'playlist_id': pid1,
            'playlist_name': info1['name'],
            'reason': f'Exact duplicate of "{info2["name"]}"',
            'tracks_lost': 0,
            'alternative': info2['name']
        })

# 2. Subsets - safe to delete (all tracks are in superset)
for subset_pid, superset_pid, subset_size, superset_size in subsets:
    if subset_pid not in safe_to_delete:
        subset_info = playlist_info[subset_pid]
        superset_info = playlist_info[superset_pid]
        safe_to_delete.add(subset_pid)
        consolidation_suggestions.append({
            'action': 'delete',
            'playlist_id': subset_pid,
            'playlist_name': subset_info['name'],
            'reason': f'All {subset_size} tracks are in "{superset_info["name"]}" ({superset_size} tracks)',
            'tracks_lost': 0,
            'alternative': superset_info['name']
        })

# 3. High overlap - suggest merging
for pid1, pid2, jaccard, overlap1, overlap2 in high_overlap:
    info1 = playlist_info[pid1]
    info2 = playlist_info[pid2]
    
    # Determine which to keep (prefer larger or better name)
    if info1['track_count'] > info2['track_count']:
        keep_pid, delete_pid = pid1, pid2
        keep_info, delete_info = info1, info2
        missing_tracks = len(playlist_track_sets[pid2] - playlist_track_sets[pid1])
    elif info2['track_count'] > info1['track_count']:
        keep_pid, delete_pid = pid2, pid1
        keep_info, delete_info = info2, info1
        missing_tracks = len(playlist_track_sets[pid1] - playlist_track_sets[pid2])
    else:
        # Same size, keep the one with longer name (usually more descriptive)
        if len(info1['name']) >= len(info2['name']):
            keep_pid, delete_pid = pid1, pid2
            keep_info, delete_info = info1, info2
            missing_tracks = len(playlist_track_sets[pid2] - playlist_track_sets[pid1])
        else:
            keep_pid, delete_pid = pid2, pid1
            keep_info, delete_info = info2, info1
            missing_tracks = len(playlist_track_sets[pid1] - playlist_track_sets[pid2])
    
    if delete_pid not in safe_to_delete:
        safe_to_delete.add(delete_pid)
        consolidation_suggestions.append({
            'action': 'merge',
            'playlist_id': delete_pid,
            'playlist_name': delete_info['name'],
            'reason': f'{jaccard*100:.1f}% similar to "{keep_info["name"]}"',
            'tracks_lost': missing_tracks,
            'alternative': f'Merge into "{keep_info["name"]}" (add {missing_tracks} missing tracks)'
        })

print(f"✅ Identified {len(safe_to_delete)} playlists that can be safely deleted/merged")
print(f"📊 Total consolidation suggestions: {len(consolidation_suggestions)}")


In [ ]:
# Display comprehensive deletion/consolidation recommendations
if consolidation_suggestions:
    print("\n" + "=" * 80)
    print("📋 CONSOLIDATION RECOMMENDATIONS")
    print("=" * 80)
    
    df = pd.DataFrame(consolidation_suggestions)
    df = df.sort_values(['tracks_lost', 'playlist_name'])
    
    # Separate by action type
    delete_actions = df[df['action'] == 'delete']
    merge_actions = df[df['action'] == 'merge']
    
    if len(delete_actions) > 0:
        print(f"\n🗑️  SAFE TO DELETE ({len(delete_actions)} playlists - 0 tracks lost):")
        print("-" * 80)
        for _, row in delete_actions.iterrows():
            print(f"   • {row['playlist_name']}")
            print(f"     → {row['reason']}")
            print(f"     → Keep: {row['alternative']}")
            print()
    
    if len(merge_actions) > 0:
        print(f"\n🔀 MERGE RECOMMENDATIONS ({len(merge_actions)} playlists):")
        print("-" * 80)
        for _, row in merge_actions.iterrows():
            print(f"   • {row['playlist_name']}")
            print(f"     → {row['reason']}")
            if row['tracks_lost'] > 0:
                print(f"     → ⚠️  {row['tracks_lost']} unique tracks would need to be added to {row['alternative']}")
            else:
                print(f"     → ✅ No tracks lost - safe to merge")
            print()
    
    # Summary statistics
    total_tracks_lost = df['tracks_lost'].sum()
    zero_loss = len(df[df['tracks_lost'] == 0])
    
    print("\n" + "=" * 80)
    print("📊 SUMMARY")
    print("=" * 80)
    print(f"   Total playlists recommended for deletion/merge: {len(df)}")
    print(f"   Playlists with zero track loss: {zero_loss}")
    print(f"   Total unique tracks that would need to be added: {total_tracks_lost}")
    print(f"   Current total playlists: {len(playlists)}")
    print(f"   After consolidation: {len(playlists) - len(df)} playlists")
    print(f"   Reduction: {len(df)} playlists ({100*len(df)/len(playlists):.1f}%)")
else:
    print("\n✅ No consolidation recommendations - your library is well organized!")


## 7️⃣ Detailed Track-Level Analysis

For merge recommendations, let's see exactly which tracks would need to be added.


In [ ]:
# For merge actions, show which tracks need to be added
merge_details = []

for suggestion in consolidation_suggestions:
    if suggestion['action'] == 'merge' and suggestion['tracks_lost'] > 0:
        delete_pid = suggestion['playlist_id']
        delete_tracks = playlist_track_sets[delete_pid]
        
        # Find the playlist to merge into
        keep_name = suggestion['alternative'].split('"')[1] if '"' in suggestion['alternative'] else suggestion['alternative']
        keep_pid = None
        for pid, info in playlist_info.items():
            if info['name'] == keep_name:
                keep_pid = pid
                break
        
        if keep_pid:
            keep_tracks = playlist_track_sets[keep_pid]
            missing_tracks = delete_tracks - keep_tracks
            
            if missing_tracks:
                # Get track names
                missing_track_ids = list(missing_tracks)[:10]  # Show first 10
                tracks_df = analyzer.tracks_all[analyzer.tracks_all['track_id'].isin(missing_track_ids)]
                track_names = tracks_df[['name']].values.flatten().tolist() if len(tracks_df) > 0 else []
                
                merge_details.append({
                    'Delete': suggestion['playlist_name'],
                    'Merge Into': keep_name,
                    'Missing Tracks': len(missing_tracks),
                    'Sample Tracks': ', '.join(track_names[:5]) if track_names else 'N/A'
                })

if merge_details:
    print("📋 Detailed Merge Analysis (tracks that need to be added):")
    print("=" * 80)
    df = pd.DataFrame(merge_details)
    for _, row in df.iterrows():
        print(f"\n🗑️  Delete: {row['Delete']}")
        print(f"   → Merge into: {row['Merge Into']}")
        print(f"   → Add {row['Missing Tracks']} tracks")
        if row['Sample Tracks'] != 'N/A':
            print(f"   → Sample: {row['Sample Tracks']}...")
else:
    print("✅ All merge recommendations have zero track loss!")


## 8️⃣ Reorganization Strategy

Based on the analysis, here's a suggested reorganization of your library.


In [ ]:
# Build reorganization plan
reorganization_plan = {
    'delete': [],
    'merge': [],
    'keep': []
}

# Categorize all playlists
all_playlist_ids = set(playlist_track_sets.keys())
to_delete_ids = safe_to_delete
to_keep_ids = all_playlist_ids - to_delete_ids

# Build merge groups
merge_groups = defaultdict(list)
for suggestion in consolidation_suggestions:
    if suggestion['action'] == 'merge':
        keep_name = suggestion['alternative'].split('"')[1] if '"' in suggestion['alternative'] else suggestion['alternative']
        merge_groups[keep_name].append(suggestion['playlist_name'])

# Organize recommendations
for suggestion in consolidation_suggestions:
    if suggestion['action'] == 'delete':
        reorganization_plan['delete'].append({
            'name': suggestion['playlist_name'],
            'reason': suggestion['reason'],
            'alternative': suggestion['alternative']
        })
    elif suggestion['action'] == 'merge':
        reorganization_plan['merge'].append({
            'name': suggestion['playlist_name'],
            'reason': suggestion['reason'],
            'merge_into': suggestion['alternative'],
            'tracks_to_add': suggestion['tracks_lost']
        })

# Keep all others
for pid in to_keep_ids:
    info = playlist_info[pid]
    reorganization_plan['keep'].append({
        'name': info['name'],
        'track_count': info['track_count']
    })

print("=" * 80)
print("📋 REORGANIZATION PLAN")
print("=" * 80)

print(f"\n🗑️  DELETE ({len(reorganization_plan['delete'])} playlists):")
print("-" * 80)
for item in reorganization_plan['delete']:
    print(f"   • {item['name']}")
    print(f"     → {item['reason']}")
    print(f"     → Keep: {item['alternative']}")
    print()

print(f"\n🔀 MERGE ({len(reorganization_plan['merge'])} playlists):")
print("-" * 80)
for item in reorganization_plan['merge']:
    print(f"   • {item['name']}")
    print(f"     → {item['reason']}")
    if item['tracks_to_add'] > 0:
        print(f"     → ⚠️  Add {item['tracks_to_add']} tracks to: {item['merge_into']}")
    else:
        print(f"     → ✅ {item['merge_into']}")
    print()

print(f"\n✅ KEEP ({len(reorganization_plan['keep'])} playlists):")
print("-" * 80)
# Sort by track count
keep_sorted = sorted(reorganization_plan['keep'], key=lambda x: x['track_count'], reverse=True)
for item in keep_sorted[:20]:  # Show top 20
    print(f"   • {item['name']} ({item['track_count']} tracks)")
if len(keep_sorted) > 20:
    print(f"   ... and {len(keep_sorted) - 20} more playlists")

print("\n" + "=" * 80)
print("📊 REORGANIZATION SUMMARY")
print("=" * 80)
print(f"   Current playlists: {len(playlists)}")
print(f"   Delete: {len(reorganization_plan['delete'])}")
print(f"   Merge: {len(reorganization_plan['merge'])}")
print(f"   Keep: {len(reorganization_plan['keep'])}")
print(f"   Final count: {len(reorganization_plan['keep']) + len(merge_groups)}")
print(f"   Reduction: {len(reorganization_plan['delete']) + len(reorganization_plan['merge'])} playlists")


In [ ]:
# Export recommendations to CSV
export_df = pd.DataFrame(consolidation_suggestions)

# Add track counts using playlist_id
export_df['track_count'] = export_df['playlist_id'].apply(
    lambda pid: playlist_info.get(pid, {}).get('track_count', 0)
)

# Reorder columns
export_df = export_df[['playlist_name', 'track_count', 'action', 'reason', 'tracks_lost', 'alternative']]

# Save to CSV
output_file = DATA_DIR / 'playlist_consolidation_recommendations.csv'
export_df.to_csv(output_file, index=False)

print(f"✅ Recommendations exported to: {output_file}")
print(f"\n📊 Summary:")
print(f"   Total recommendations: {len(export_df)}")
print(f"   Safe deletions (0 tracks lost): {len(export_df[export_df['tracks_lost'] == 0])}")
print(f"   Merge recommendations: {len(export_df[export_df['action'] == 'merge'])}")
print(f"\n💡 Next steps:")
print(f"   1. Review the CSV file: {output_file}")
print(f"   2. Manually verify recommendations")
print(f"   3. Delete/merge playlists in Spotify")
print(f"   4. Re-run sync to update your library")


## ✅ Done!

**Summary:**
- ✅ Identified redundant playlists using multiple similarity metrics
- ✅ Found playlists safe to delete (zero track loss)
- ✅ Suggested merge strategies for high-overlap playlists
- ✅ Created reorganization plan
- ✅ Exported recommendations to CSV

**Next Steps:**
1. Review the recommendations in the CSV file
2. Manually verify each suggestion
3. Delete/merge playlists in Spotify
4. Re-run `01_sync_data.ipynb` to update your library
5. Re-run this notebook to verify cleanup

**Note:** Always verify recommendations manually before deleting playlists!
